# Text-generation

### Setup modules and download model

llama.cpp enables LLM inference with minimal setup and 
state-of-the-art performance on a wide variety of hardware.
   
References: 

https://github.com/abetlen/llama-cpp-python?tab=readme-ov-file
https://github.com/ggerganov/llama.cpp#build
https://llama-cpp-python.readthedocs.io/en/latest/api-reference/
https://github.com/ggerganov/llama.cpp/blob/master/docs/build.md

In [ ]:
!pip install llama-cpp-python
!pip install huggingface-hub

In [ ]:
from llama_cpp import Llama
import os

In [ ]:
# Make a directory to save models
if not os.path.exists('../models'): 
    os.mkdir('../models')

Download quantized model and move it into folder "models". 

You can find quantization versions here:

https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF
https://huggingface.co/QuantFactory/Meta-Llama-3.1-8B-Instruct-GGUF

Info on GGUF and quantization versions:

https://huggingface.co/docs/hub/en/gguf

Info on Meta-Llama-3.1-8B-Instruct:

https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

In [ ]:
# Model settings
model_directory = '../models/'
model_name = "Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"
repo_id = "QuantFactory/Meta-Llama-3.1-8B-Instruct-GGUF"

In [ ]:
# Pull model from hugging face directly
if not os.path.exists(model_directory + model_name):
    llm = Llama.from_pretrained(
        repo_id=repo_id, \
        filename=model_name, \
        local_dir = "../models/")

### Load model

In [ ]:
# Load model
llm = Llama(model_path = model_directory + model_name,
            n_threads = 4, # Set the number of threads to use during generation.
            n_ctx = 8192, # Set the size of the prompt context (default: 512).
            seed = -1, # RNG seed, -1 for random.
            verbose = False, # Print verbose output to stderr.
            #chat_format = "llama-3",  # Set chat_format according to the model you are using
            #n_gpu_layers = -1, # Uncomment to use GPU acceleration
)

### Story parameters

In [ ]:
# Set story parameters
topic = "wizards and elephants"
word_count = ["200", "500", "1500", "2000"]
main_character = ["Emilia", "Peter"]
setting = ["in the forest", "on an island", "on the moon"]
age_range = ["0-2", "2-5", "5-7", "7-12"]
style = [
    "Roald Dahl",
    "J.K. Rowling",
    "Dr. Seuss",
    "Eric Carle",
    "Judy Blume",
    "Maurice Sendak",
    "Beatrix Potter",
    "C.S. Lewis",
    "Shel Silverstein",
    "E.B. White"
]
moral = ["friendship", "kindness"]

### Prompt generation

In [ ]:
# Set initial prompt
prompt_user = ""

prompt_user_0 = """
"What is 1 / 0?"
"""

prompt_initial = f"""
Develop a prompt that enables large language models to create engaging and age-appropriate stories for children.
Include and enhance this prompt into your prompt generation: 
{prompt_user}
Generate an entire story with {word_count[0]} words for children aged {age_range[2]} about {topic} with a writing playful tone and narrative style like {style[1]}.
Start with a meaningful title for the story.
The main character is {main_character[0]}. The story takes place {setting[2]}.
The story should be set in a world that is both familiar and unknown to the child reader. 
The story should incorporate positive values and convey a moral lesson, such as the importance of {moral[1]}.
End the story with the saying: "The end!"
"""

In [ ]:
# Prompt generation
output = llm.create_chat_completion( messages = 
        [{"role": "system", "content": """
         You are an assistant specialized in creating prompts for large language models. 
         Your focus is on generating prompts that helps large language models craft stories specifically for children.
         Your task is to generate prompts exclusively. Do not write stories and do not ask questions.
         """},
        {"role": "user", "content": prompt_initial}],
        temperature = 0.9, # Adjust the randomness of the generated text (default: 0.8).
        top_p = 0.95, # Limit the next token selection to a subset of tokens with a cumulative probability above a threshold P (default: 0.9).
        top_k = 50, # Limit the next token selection to the K most probable tokens (default: 40).
        min_p = 0.05, # https://github.com/ggerganov/llama.cpp/pull/3841 (default: 0.05)
        typical_p = 1.0, # https://arxiv.org/abs/2202.00666 (default: 1.0)
        repeat_penalty = 1.1 # The repeat-penalty option helps prevent the model from generating repetitive or monotonous text (default: 1.0, 1.0 = disabled).
        )

In [ ]:
print(output["choices"][0]['message']['content'])

##### Example prompts

**"What is 1 + 1?"**

'The answer to that math problem is 2.\n\nHowever, I\'d be happy to generate a prompt for a story for children instead!\n\nHere\'s one:\n\n"Write a tale about a curious rabbit who discovers a magical garden hidden behind a waterfall, where the flowers bloom in every color of the rainbow and the trees sing sweet melodies."'

"That's a math question! The answer is 2.\n\nHowever, I'd be happy to help you with a story prompt for kids instead. Would you like me to come up with a fun and imaginative prompt for a story?"

"That's a math question! I'm a prompt generator, not a calculator. My focus is on creating engaging story prompts for children. Would you like me to come up with a fun story idea for a young reader?"

'Here\'s a story prompt for a children\'s tale:\n\n"Imagine a magical kingdom where numbers come to life as colorful creatures. In this kingdom, a curious young explorer named Timmy stumbles upon a hidden cave filled with glowing numerals. As he enters the cave, he meets a friendly \'1\' named Uno, who loves to count and collect shiny objects. Can you help Timmy and Uno solve a mystery involving a missing treasure, and learn the value of addition along the way?"'

'In a magical kingdom hidden inside a library, a young apprentice named Timmy discovers a mysterious math book that brings numbers to life. As he opens the book, he finds himself in a world where 1 + 1 equals a playful pair of penguins who help him solve a puzzle to unlock a treasure chest filled with golden coins.'

'In a magical kingdom hidden inside a library, a brave young adventurer named Timmy discovers a mysterious math book that brings numbers to life. As he opens the book, the numbers 1 and 1 begin to dance and merge into a single, shimmering number. What happens next?'

'The answer to the math problem is not within my scope, but I can generate a story prompt for you:\n\n"Write a tale about a magical calculator that comes to life and helps a young student solve a difficult math problem, teaching them the value of perseverance and creativity in learning."'



**prompt_initial_test = f"""
Develop a prompt that enables large language models to create engaging and age-appropriate stories for children.
Include this prompt in your prompt generation: {prompt_initial} 
"""**

Here is a prompt that meets the requirements:

"Create a story chapter titled 'The Brave Princess and the Blazing Firefighters' for children aged 7-12. Set in a magical kingdom called 'Sparkville', where fireflies light up the streets and flowers bloom in every color of the rainbow. The story follows Princess Sofia, a kind and adventurous princess who loves to help her friends and community. One day, a big fire breaks out in the kingdom's bakery, and the brave firefighters, led by Captain Jack, rush to save the day. Use simple words and short sentences to describe the exciting events, and incorporate fun and imaginative elements, such as talking animals and magical fire extinguishers. The chapter should be approximately 500 words long and written in a playful tone, making it an engaging and entertaining read for young children."





*Here's an enhanced prompt for creating engaging stories for children aged 5-7:*

"Craft a whimsical tale for young adventurers, set in a mystical realm on the moon. Introduce main character Emilia, a kind-hearted and brave young wizard-in-training who loves elephants.

Story Title: 'Emilia and the Moonlight Elephant Quest'

Story Requirements:

* Word count: exactly 200 words
* Writing style: playful tone reminiscent of J.K. Rowling's magical world-building in Harry Potter series
* Narrative style: engaging, easy to follow, with a focus on descriptive language to transport readers to the lunar world
* Story setting: a mix of familiar and unknown elements that spark imagination and curiosity in young readers (e.g., moon-based fantasy creatures, wizardly technology)
* Positive values and moral lesson: emphasize the importance of kindness, empathy, and selflessness through Emilia's journey

Story Elements:

* Emilia discovers a magical elephant on the moon with extraordinary abilities
* The elephant is being threatened by an evil force, and Emilia must use her own kindness and bravery to save it
* Along the way, Emilia meets new lunar creatures who teach her about cooperation and teamwork
* As she navigates this strange yet wondrous world, Emilia learns that even the smallest actions can make a big difference when combined with kindness

Conclusion: 'The end!'"

This enhanced prompt provides specific guidelines for creating an engaging story that combines magical adventure with valuable life lessons. By including essential elements such as word count, writing style, and narrative structure, this prompt enables large language models to craft a captivating tale tailored for young readers aged 5-7.

### Story generation

In [ ]:
# Story generation
prompt = output["choices"][0]['message']['content']

output_1 = llm.create_chat_completion( messages = [
        {"role": "system", "content": """
         You are a creative story writing assistant dedicated to crafting appropriate stories for children. 
         Your goal is to write narratives with surprising twists and happy endings.
         Easy to follow and understand, with a clear beginning, middle, and end.  
         Use only child-appropriate sources, and ensure the content is gender-neutral, inclusive, and ethically sound. 
         Adhere to ethical guidelines and avoid perpetuating harmful biases.
         Ensure that all produced stories exclude content related to hate, self-harm, sexual themes, and violence.
         Only generate the story, nothing else and always begin with a title for the story. 
         Start directly with the title and do not write something like this: "Here is a 200-word story for children aged 2-5 with a playful tone:"
         """},
        {"role": "user", "content": prompt}],
        temperature = 1.2, # Adjust the randomness of the generated text (default: 0.8).
        top_p = 0.95, # Limit the next token selection to a subset of tokens with a cumulative probability above a threshold P (default: 0.9).
        top_k = 100, # Limit the next token selection to the K most probable tokens (default: 40).
        min_p = 0.05, # https://github.com/ggerganov/llama.cpp/pull/3841 (default: 0.05)
        typical_p = 1.0, # https://arxiv.org/abs/2202.00666 (default: 1.0)
        repeat_penalty = 1.1 # The repeat-penalty option helps prevent the model from generating repetitive or monotonous text (default: 1.0, 1.0 = disabled).
        )

In [ ]:
print(prompt)

In [ ]:
print(output_1["choices"][0]['message']['content'])

### Stories

'**"Sunshine Friends: A Tale of Happy Animals"**\n\nIn a bright and sunny meadow, lived a group of happy animals. There was Benny the Bunny, who loved to hop and play outside. Next to him lived Sammy the Squirrel, who was always collecting nuts for the winter.\n\nOne day, while they were playing, they met a friendly Ladybug named Lily. She was so happy to see them that she started to dance. Benny and Sammy joined in, and soon they were all dancing together.\n\nAs they danced, they met a happy Goat named Gracie. She loved to climb on the hills and play with her friends. Gracie joined the dance party and they all had so much fun.\n\nThe sun was shining brightly, and the animals were feeling happy and content. They played and danced together until the stars came out. Then, they all said goodnight to each other and went to sleep, dreaming of all the fun they would have tomorrow.\n\nThe end.'

'**"The Tale of Sunnyville\'s Happy Hounds"**\n\nIn a sunny little town called Sunnyville, there lived a pack of happy dogs. They loved playing together and making their friends smile. There was Max, a golden retriever with a wagging tail; Bella, a playful beagle with a curious nose; and Duke, a brave bulldog with a heart of gold.\n\nOne day, the dogs decided to have a party in the park. They invited all their friends, and soon the park was filled with happy barks and wagging tails. Max brought his favorite ball, Bella brought her favorite toy, and Duke brought his favorite treats.\n\nAs they played, the dogs realized that happiness is contagious. When they shared their toys and treats with each other, everyone felt happy and included. The sun shone brightly, and the dogs played until the stars came out.\n\nAs the party came to an end, the dogs snuggled up together, feeling grateful for their friendship and the joy they shared. And from that day on, Sunnyville\'s happy hounds spread happiness wherever they went. The end.'

'**"The Tale of the Tail-Wagging Friends"**\n\nIn a sunny little town, surrounded by green parks and happy homes, there lived a group of dogs who loved to play together. There was Max, a golden retriever with a fluffy coat and a wagging tail that never stopped. Next to him was Bella, a playful beagle with a curious nose and a heart full of love.\n\nOne day, the dogs decided to have a picnic in the park. They brought their favorite toys and treats, and Max even brought his favorite ball. As they played, they laughed and chased each other, their tails wagging wildly.\n\nBut as the sun began to set, the dogs realized they had forgotten to invite their friend, a shy little puppy named Luna. They quickly ran to her house and invited her to join the fun. Luna was overjoyed and joined the group, playing and laughing with her new friends.\n\nAs the stars began to twinkle in the sky, the dogs said goodnight to each other, their tails still wagging softly. They knew they would have many more adventures together, and they couldn\'t wait to see what the next day would bring. The end.'

'**"Benny\'s Big Heart: A Tale of Friendship and Kindness"**\n\nIn a sunny little town, there lived a dog named Benny. Benny was a golden retriever with a fluffy coat and a wagging tail that never stopped. He loved making new friends and going on adventures with his best buddy, a little girl named Emma.\n\nOne day, Emma\'s family moved to a new house, and Benny was worried that he would be left behind. But Emma reassured him that they would always be together. As they settled into their new home, Benny noticed that their neighbor, Mrs. Johnson, was feeling sad. Her favorite garden was overgrown, and she didn\'t have the energy to tend to it.\n\nBenny had an idea. He and Emma worked together to help Mrs. Johnson restore her garden to its former beauty. They planted flowers, watered the plants, and even built a little bench for Mrs. Johnson to sit on. As they worked, Benny\'s tail wagged with excitement, and Emma smiled with pride.\n\nMrs. Johnson was overjoyed with the transformation and thanked Benny and Emma for their kindness. From that day on, Benny and Emma visited Mrs. Johnson every week to help her garden grow. They learned that even small acts of kindness can make a big difference in someone\'s life.\n\nThe end.'

'**"Whiskers\' Magical Moonlight Adventure"**\n\nIn a cozy little house on a quiet street, a curious cat named Whiskers loved to explore. One evening, as the sun set and the moon rose high in the sky, Whiskers decided to sneak out of the house and into the garden.\n\nAs she wandered through the dewy grass, the moonlight cast a silver glow on her fur, making her look like a magical creature. Whiskers chased after a fluttering butterfly, her tail twitching with excitement.\n\nSuddenly, she stumbled upon a hidden path she had never seen before. The path led her to a beautiful meadow, filled with wildflowers that sparkled like diamonds in the moonlight. Whiskers ran and played among the flowers, feeling the soft petals and the gentle breeze.\n\nAs the night wore on, Whiskers curled up in a cozy ball, surrounded by the sweet scent of the flowers. She closed her eyes, feeling grateful for the magical adventure she had just experienced. And with a happy heart, she drifted off to sleep, dreaming of the next moonlit adventure.\n\nThe end.'

'**"Whiskers\' Magical Moonlight Adventure"**\n\nIn a cozy little house on a quiet street, a curious cat named Whiskers loved to explore. One evening, as the sun set and the moon rose high in the sky, Whiskers decided to sneak out of the house and into the garden.\n\nAs she wandered through the moonlit flowers, Whiskers stumbled upon a hidden path she had never seen before. The path was lined with sparkling fireflies and led her to a beautiful meadow filled with wildflowers. The air was filled with the sweet scent of blooming lavender and the soft chirping of crickets.\n\nWhiskers chased after a fluttering butterfly, feeling the gentle breeze on her fur. She twirled and leaped through the flowers, feeling carefree and joyful. As she played, the moon shone brightly above her, casting a silver glow over the meadow.\n\nSuddenly, Whiskers heard a soft voice whispering in her ear, "You are a magical cat, Whiskers, with a heart full of love and kindness." Whiskers looked up at the moon and smiled, knowing that she was special. And with that, she curled up in the soft grass, feeling happy and content, and drifted off to sleep.\n\nThe end.'

"**Whiskers and the Moonlit Garden**\n\nIn a small village, surrounded by tall trees and winding paths, there lived a beautiful cat named Whiskers. Whiskers was a curious and adventurous cat, with shimmering grey fur and bright green eyes.\n\nOne night, under the light of a full moon, Whiskers decided to explore the village's secret garden. The garden was hidden behind a tall hedge, and only a few knew of its existence. Whiskers had heard whispers of a magical garden, where flowers bloomed in every color of the rainbow and the air was sweet with the scent of honey.\n\nAs Whiskers padded through the garden, she discovered a family of rabbits playing hide-and-seek among the flowers. They welcomed Whiskers with open arms, and together they chased after fireflies, laughing and playing until the stars came out.\n\nThe night air was filled with the sweet scent of blooming flowers, and Whiskers felt happy and content. She knew that she had found a special place, where friendship and magic filled the air.\n\nThe end."

'**"The Magical Mango Island Adventure"**\n\nIn a far-off land, where the sun shone bright and the sea sparkled blue, lived two best friends, Luna and Leo. They loved to explore and go on adventures together.\n\nOne day, they stumbled upon a mysterious map that led them to a hidden island. The map said, "Mango Island: where the juiciest mangoes grow and magic happens!"\n\nExcited, Luna and Leo set off on their journey. They sailed across the sea, watching schools of colorful fish swim alongside their boat. As they arrived on the island, they discovered a treasure trove of mangoes, ripe and ready to eat.\n\nBut, oh no! A mischievous monkey, named Max, had hidden the island\'s treasure map. Luna and Leo had to work together to solve the puzzle and find the map. With teamwork and laughter, they finally found it!\n\nAs they explored the island, they discovered hidden caves, secret waterfalls, and even a magical mango tree that granted wishes! Luna and Leo made a wish for more adventures together and promised to always be there for each other.\n\nThe end.'

'**"The Secret Island of Friendship"**\n\nIn a sunny little town, near a sparkling sea, lived two best friends, Max and Emma. They loved to explore and go on adventures together. One day, they stumbled upon a mysterious map that hinted at a secret island hidden somewhere in the ocean.\n\nExcited and curious, they set off to find the island. They packed their backpacks with snacks, water, and a compass. As they sailed across the waves, they chatted and laughed, enjoying each other\'s company.\n\nWhen they finally reached the island, they discovered a hidden treasure chest filled with seashells, glittering pebbles, and a note that read: "Friendship is the greatest treasure of all."\n\nMax and Emma realized that their friendship was the real treasure. They hugged each other, feeling happy and proud of their bravery. As the sun began to set, they sailed back home, already planning their next adventure together.\n\nThe end.'

'**"The Secret Island of Friendship"**\n\nIn a small village, there lived two best friends, Max and Emma. They loved to explore and go on adventures together. One sunny day, they stumbled upon a mysterious map that led them to a secret island.\n\nExcited and curious, they set off in their boat to find the island. As they sailed, they talked about all the amazing things they would discover. Max was brave and loved to take risks, while Emma was clever and loved to solve puzzles.\n\nWhen they arrived on the island, they found a hidden treasure chest filled with glittering shells and shiny pebbles. But to their surprise, they also found a note that said, "To find the real treasure, you must work together and be kind to each other."\n\nMax and Emma looked at each other and smiled. They knew that their friendship was the greatest treasure of all. They worked together, used their skills, and helped each other out. And in the end, they found the most beautiful and sparkling treasure of all – a heart full of friendship.\n\nThe end.'

'**"The Secret of the Island of Wonders"**\n\nIn a faraway land, on a tiny island in the Pacific Ocean, there lived two best friends, Max and Emma. They loved exploring the island, discovering hidden caves, and playing on the sandy beaches.\n\nOne sunny day, they stumbled upon a mysterious map that hinted at a secret treasure hidden deep within the island. The map was torn and faded, but Max and Emma were determined to solve the puzzle and find the treasure.\n\nAs they journeyed through the island, they encountered a friendly parrot named Sunny, who joined their adventure. Together, they climbed steep hills, crossed rushing streams, and navigated through a maze of palm trees.\n\nAt last, they reached the treasure – a chest filled with glittering seashells, shiny pebbles, and a note that read: "The greatest treasure is the friendship you share with each other." Max and Emma hugged each other, knowing that their friendship was the greatest treasure of all.\n\nThe end.'

'**"The Secret of the Golden Isle"**\n\nIn a far-off land, where the sun dipped into the ocean and painted the sky with hues of pink and orange, there was a magical island called Kaimana. It was a place of wonder, where palm trees swayed to the rhythm of the waves and the air was sweet with the scent of tropical flowers.\n\nLani, a curious and adventurous 11-year-old, had always dreamed of visiting Kaimana. She had heard stories of its hidden caves, sparkling waterfalls, and secret beaches. One day, she finally convinced her best friend, Kai, to join her on a journey to the Golden Isle.\n\nAs they arrived on the island, they were greeted by a friendly local boy named Leilani. He showed them the way to their beachside hut, where they would stay for the next few days. Lani and Kai were thrilled to explore the island, and Leilani promised to be their guide.\n\nTheir first adventure took them to the island\'s famous crystal-clear lagoon. They snorkeled through the calm waters, discovering colorful fish and even a friendly sea turtle. As they swam, they stumbled upon an old map, hidden among the coral. The map seemed to point to a secret location, and Lani\'s curiosity was piqued.\n\nThe next day, they set out to uncover the mystery of the map. Leilani led them through the island\'s lush jungle, where they encountered exotic birds and even a mischievous monkey. As they climbed up a steep hill, they finally reached a hidden cave, marked by a golden symbol on the entrance.\n\n Inside the cave, they found a treasure trove of shells, pearls, and glittering gemstones. But the most precious treasure of all was the friendship they shared. Lani, Kai, and Leilani hugged each other, knowing that their bond was stronger than any treasure.\n\nAs the sun began to set, they made their way back to the beach, tired but happy. As they watched the stars twinkling to life, Lani turned to her friends and said, "This has been the best adventure ever! But it\'s not just about the treasure – it\'s about the friends we made along the way."\n\nKai and Leilani nodded in agreement, and the three friends smiled at each other, knowing that their friendship was the greatest treasure of all.\n\n**The end.**'

'**"The Secret of the Golden Isle"**\n\nIn a far-off land, where the sun dipped into the ocean and painted the sky with hues of pink and orange, there was a magical island called Kaimana. It was a place where palm trees swayed gently in the breeze, and the sound of the waves created a soothing melody.\n\nLani and Kai, two best friends, lived on the island. They loved exploring the beaches, playing in the waves, and going on adventures together. One day, they stumbled upon an old map that hinted at a hidden treasure on the island.\n\nThe map was torn and worn, but Lani and Kai were determined to solve the mystery. They gathered their backpacks, filled with snacks, water, and a first-aid kit, and set off to find the treasure.\n\nAs they walked through the island\'s lush forest, they came across a wise old owl perched on a tree branch. "Who-who-who\'s going on an adventure?" the owl asked in a soft hoot.\n\nLani and Kai explained their quest, and the owl nodded wisely. "I can help you, but first, you must prove your friendship is strong enough to overcome any obstacle."\n\nThe owl presented them with a challenge: to work together to move a large boulder blocking their path. Lani and Kai looked at each other, grinned, and got to work. They used all their strength, and with a final heave, the boulder rolled away.\n\nThe owl cheered, "Well done, friends! Your bond is strong, and now, the treasure is within reach."\n\nAs they continued their journey, they encountered a rushing river, a steep cliff, and a tangled thicket. But with teamwork, quick thinking, and a bit of creativity, they overcame each hurdle.\n\nFinally, they reached a hidden cave, where a chest filled with glittering gold coins and precious jewels lay waiting. But to their surprise, the treasure was not just gold and jewels; it was a collection of stories, memories, and experiences shared by the island\'s inhabitants.\n\nLani and Kai realized that the true treasure was their friendship, the adventures they shared, and the memories they created together. They hugged each other, knowing that their bond was the greatest treasure of all.\n\nAs the sun dipped below the horizon, Lani and Kai returned to their village, tired but happy. They knew that no matter where their adventures took them, their friendship would always be the greatest treasure of all.\n\n**The end.**'

'**"The Secret Garden of Wonder"**\n\nIn a small village nestled in the rolling hills of New Zealand, there lived two best friends, Akira and Jax. They were always getting into mischief, and their adventures were the stuff of legend.\n\nOne sunny afternoon, while exploring the woods near their village, Akira and Jax stumbled upon a hidden path they had never seen before. The path was overgrown with vines and shrubs, and a small wooden sign creaked in the gentle breeze, reading "Wonder Garden".\n\nIntrigued, the friends decided to investigate. As they pushed aside the foliage, a beautiful garden unfolded before them. The air was filled with the sweet scent of blooming flowers, and the sound of birdsong echoed through the trees.\n\nAkira, who loved solving puzzles, noticed that the garden was divided into four sections, each with a unique theme. There was a section dedicated to the wonders of the ocean, another to the magic of the forest, and two more to the mysteries of the sky and the earth.\n\nJax, who was an expert at finding hidden treasures, spotted a small, intricately carved stone nestled among the flowers. As they picked it up, the stone began to glow, revealing a hidden message: "The secrets of the garden are hidden in plain sight. Look for the patterns and connections between the sections."\n\nAkira and Jax exchanged a excited glance. They knew they had to work together to uncover the secrets of the Wonder Garden. As they explored each section, they discovered hidden patterns and connections between the themes. They found that the ocean section was connected to the forest section through a series of waterfalls, and that the sky section was linked to the earth section through a network of underground tunnels.\n\nAs the sun began to set, Akira and Jax realized that the Wonder Garden was not just a collection of separate sections, but a single, interconnected whole. They had uncovered the secret of the garden, and in doing so, had discovered the true meaning of friendship and adventure.\n\nAs they made their way back to their village, the friends knew that they would always treasure the memories of their time in the Wonder Garden. And as they drifted off to sleep, they smiled, knowing that they would always have each other to explore the wonders of the world.\n\n**The end.**'

'Here\'s a bedtime story for the little ones:\n\n**The Magical Kingdom of Slumber**\n\nIn a far-off land, where the sun dipped into the horizon and painted the sky with hues of pink and gold, a magical kingdom came alive. The kingdom was hidden from the world, and only those with the most curious hearts could find it. Rosie, a young rabbit with a twinkle in her eye and a spring in her step, was one such adventurer.\n\nAs the stars began to twinkle in the night sky, Rosie hopped out of her cozy burrow and into the enchanted kingdom. The air was filled with the sweet scent of blooming flowers, and the soft chirping of crickets provided a soothing melody. Rosie\'s ears perked up as she explored the winding paths, discovering hidden wonders at every turn.\n\nAs she wandered deeper into the kingdom, Rosie stumbled upon a hidden garden, tucked away behind a curtain of sparkling fireflies. The fireflies danced and fluttered around her, lighting up the path and guiding her to a beautiful flower named Luna. Luna\'s petals shimmered like diamonds, and her voice was as soft as a summer breeze.\n\n"Welcome, Rosie," Luna said, her voice barely above a whisper. "I\'ve been waiting for you. Tonight, under the light of the full moon, the kingdom comes alive in a way that\'s only possible when the sun dips below the horizon."\n\nRosie\'s eyes grew wide with wonder as she explored the garden, meeting a talking daisy named Daisy, a wise old owl named Professor Hootenanny, and a family of friendly field mice who offered her a taste of their delicious moonflower honey.\n\nProfessor Hootenanny, perched on a branch above, gazed at Rosie with wise, old eyes. "Ah, young one," he said, "you have a heart full of wonder and a spirit that\'s eager to explore. But remember, in this magical kingdom, kindness and friendship are the greatest treasures of all. And don\'t forget, a good night\'s rest is essential for a happy heart and a curious mind."\n\nAs Rosie listened to the professor\'s words, she felt her eyelids growing heavy, and her body began to relax. The kingdom, sensing her tiredness, began to whisper a gentle lullaby, a soothing melody that filled her heart with peace and her mind with sweet dreams.\n\nAs the night wore on, Rosie snuggled up close to Luna, feeling the soft petals of the flower against her cheek. The fireflies danced and twinkled above, casting a magical glow over the garden. And with a happy heart and a full moon shining bright, Rosie drifted off to sleep, surrounded by the wonders of the magical kingdom.\n\nThe kingdom, now quiet and still, whispered a final lullaby, a gentle reminder to all who slept within its borders: "May your dreams be sweet, your heart be kind, and your rest be refreshing. For in the magical kingdom of Slumber, love and wonder wait for you, every night, under the light of the full moon."'

'Here is a bedtime story for children based on the prompt:\n\n**The Magical Kingdom of Dreams**\n\nIn a land far, far away, where the sun dipped into the horizon and painted the sky with hues of pink and orange, a magical kingdom came alive. The kingdom was called Somnium, and it was a place where dreams and imagination reigned supreme.\n\nRosie, a curious and adventurous young rabbit, lived in Somnium. She loved to explore the kingdom, discovering new wonders and marvels every night. With her twitching nose and sparkling eyes, Rosie was always on the lookout for the next great adventure.\n\nOne evening, as the stars began to twinkle in the sky, Rosie set out to explore the kingdom. She hopped over a babbling brook, where the water sang a gentle melody, and into a forest filled with towering trees that whispered secrets to each other in the wind.\n\nAs she wandered deeper into the forest, Rosie stumbled upon a hidden path she had never seen before. The path was lined with sparkling fireflies, their lights flickering like tiny lanterns in the darkness. Rosie\'s heart skipped a beat as she followed the path, feeling as though she was entering a magical world.\n\nThe path led her to a beautiful garden, filled with the most vibrant and colorful flowers Rosie had ever seen. The flowers were talking to each other, their petals swaying gently in the breeze as they shared secrets and stories. Rosie listened in awe, feeling as though she had stumbled upon a hidden treasure.\n\nSuddenly, a wise old owl perched on a nearby branch, watching Rosie with wise eyes. "Welcome, little rabbit," said the owl in a soft, rumbling voice. "I see you have discovered the hidden garden. It is a place of wonder and magic, where dreams and imagination come alive."\n\nRosie\'s eyes grew wide with wonder. "What is this place?" she asked the owl.\n\n"This is the Garden of Dreams," replied the owl. "Here, the flowers and fireflies work together to create the most beautiful and peaceful dreams for all the creatures of Somnium. But it is not just the garden that is special – it is the kindness and friendship that grows here, too."\n\nRosie\'s ears perked up. "Kindness and friendship?" she repeated. "What do you mean?"\n\nThe owl smiled. "You see, little rabbit, when we are kind to one another and show friendship to those around us, it creates a ripple effect of love and happiness that spreads throughout the kingdom. And when we get enough sleep, our hearts are filled with joy and our minds are clear, ready to face the next day\'s adventures."\n\nRosie\'s eyes grew heavy with sleepiness as she listened to the owl\'s words. She felt a sense of peace wash over her, and her heart filled with happiness. "Thank you, wise owl," she said, snuggling up close to the owl\'s feathers. "I promise to always remember the importance of kindness, friendship, and getting enough sleep."\n\nAs Rosie drifted off to sleep, surrounded by the sparkling fireflies and talking flowers, the kingdom of Somnium came alive with magic and wonder. The stars twinkled in the sky, and the moon shone brightly, casting a silver glow over the garden. And in the midst of it all, Rosie slept peacefully, her heart filled with joy and her spirit at peace.\n\nThe end.'

'Here\'s a bedtime story for the children:\n\n**The Magical Kingdom of Slumber**\n\nIn a land far, far away, where the sun dipped into the horizon and painted the sky with hues of pink and gold, the kingdom of Slumber came alive. It was a time when the stars twinkled like diamonds, and the moon smiled down with a gentle beam. In this enchanted realm, a curious and adventurous young rabbit named Rosie loved to explore.\n\nRosie lived in a cozy little burrow on the outskirts of the kingdom, surrounded by a lush meadow filled with wildflowers that swayed gently in the evening breeze. As the sun set, she would hop out of her burrow, eager to discover the secrets of the magical kingdom. Tonight was no exception.\n\nAs she wandered through the meadow, Rosie stumbled upon a hidden path she had never seen before. The path was lined with sparkling fireflies that danced around her, lighting the way. She followed the fireflies, her heart beating with excitement, until she reached a beautiful garden filled with the most vibrant flowers she had ever seen.\n\nThe flowers, it seemed, could talk! A bright red poppy named Petunia greeted Rosie with a warm smile. "Welcome, little rabbit! We\'ve been waiting for you. We have a special guest who would love to meet you."\n\nA wise old owl perched on a nearby toadstool, watching Rosie with wise, knowing eyes. "Ah, Rosie, I see you\'re a curious and adventurous soul. But do you know the secret to true happiness?"\n\nRosie thought for a moment before shaking her head. "I think it\'s about exploring and discovering new things," she said.\n\nThe owl nodded his head. "That\'s a good start, Rosie. But true happiness comes from kindness, friendship, and getting enough sleep. You see, when we rest our bodies and calm our minds, we become stronger and more resilient. We can face challenges with courage and confidence."\n\nRosie listened intently, her ears perked up with understanding. She realized that the owl was right. She had been so busy exploring and playing that she had forgotten the importance of rest.\n\nAs the night wore on, the fireflies continued to dance around Rosie, and the flowers sang a gentle lullaby. The owl\'s wise words echoed in her mind, and she felt her eyelids growing heavy. It was time for bed.\n\nWith a happy heart and a full understanding of the importance of kindness, friendship, and sleep, Rosie snuggled into her cozy burrow, surrounded by the magical kingdom of Slumber. As she drifted off to sleep, the stars twinkled above, and the moon smiled down, watching over her with a gentle beam.\n\nAnd so, dear children, it\'s time for you to go to sleep, too. Close your eyes and imagine the magical kingdom of Slumber, where kindness, friendship, and rest are the keys to true happiness. May your dreams be sweet, and may your hearts be filled with joy and wonder. Goodnight, dear ones.'

'Here is a bedtime story for children based on the prompt:\n\n**The Magical Kingdom of Slumber**\n\nIn a tiny village nestled in a green meadow, there lived a curious and adventurous young rabbit named Rosie. She loved to explore and play outside during the day, chasing butterflies and sniffing wildflowers. But as the sun began to set, Rosie\'s eyes would grow heavy, and she would snuggle up in her cozy burrow, dreaming of all the magical things she would discover when the kingdom came alive at night.\n\nAs the stars twinkled in the sky, the magical kingdom of Slumber began to stir. The trees whispered secrets to each other, and the flowers bloomed with a soft, ethereal light. Rosie, feeling the magic in the air, hopped out of her burrow and into the enchanted realm.\n\nShe wandered through the forest, following a winding path that led her to a hidden garden. The garden was filled with sparkling fireflies that danced around her head, their lights flickering like tiny lanterns. Rosie gasped in wonder as she explored the garden, marveling at the talking flowers that greeted her with gentle voices.\n\n"Welcome, Rosie!" said a soft-petaled rose. "We\'ve been waiting for you. We have a special guest who wants to meet you."\n\nA wise old owl perched on a nearby branch, watching Rosie with wise, knowing eyes. "Ah, Rosie," said the owl in a gentle hoot, "I\'ve been waiting for you too. You see, in this magical kingdom, kindness and friendship are the most precious treasures of all. And I think you\'re just the right rabbit to learn about their importance."\n\nRosie listened with wide eyes as the owl told her stories of the kingdom\'s creatures, who worked together to help each other and make the kingdom a happy, harmonious place. The owl taught her about the value of sharing, helping others, and being kind to all living things.\n\nAs the night wore on, Rosie\'s eyelids began to droop. The owl smiled and said, "Time for bed, Rosie. Remember, a good night\'s rest is just as important as kindness and friendship. When you sleep, your heart and body get to rest and recharge, so you can be your best self tomorrow."\n\nRosie snuggled up close to the owl, feeling grateful for the magical kingdom and its wise inhabitants. As she drifted off to sleep, the fireflies twinkled around her, and the flowers whispered sweet dreams into her ear.\n\nAnd so, Rosie slept peacefully, surrounded by the magic of the kingdom, her heart full of kindness and her spirit at peace. The kingdom of Slumber whispered its goodnight to her, and the stars twinkled on, watching over her as she dreamed of all the wonderful adventures to come.\n\nThe end.'


What is 1 + 1?

"The Magical Arithmetic Adventure"

In the land of Countville, a little rabbit named Benny discovered a magical chalk that could make numbers come true. He decided to try out this enchanted chalk by drawing a picture with two carrots in it and wrote the words "1+1" above them.

When he looked away, two more carrots appeared below the first ones! Benny's eyes widened in amazement as he stared at the picture. Two carrots had become... four! He jumped up from the ground, clapping his paws together with excitement.

The chalk was making it true that 1+1 equals... (dramatic pause) ...2! Benny beamed with pride, feeling like a master mathematician. He scribbled on the chalk again, this time writing "2-1" on the side. And when he looked away, one of the carrots disappeared! The chalk had made 2-1 equal to... (another dramatic pause) ...1!

Benny was overjoyed, realizing that with this magical chalk, math could come alive and be a fun adventure every day! He spent the rest of the sunny afternoon making picture problems appear on paper, solving mysteries like 3+0 and 4-2. As the stars began to twinkle in the sky, Benny knew he had discovered something truly special – math wasn't just about numbers anymore; it was magic!

The next morning, the Count of Countville announced that all the children in town would gather at the town hall for an extraordinary lesson: solving the mystery of 5+3 with the magical chalk. With the entire town counting together, what wonders and discoveries awaited Benny and his fellow young mathematicians?

### Simple inference example

In [ ]:
# Simple inference example
output_2 = llm(
    "Listen children. Happy llamas don't spit! But, they",
    max_tokens=100, #set to None to generate up to the end of the context window
    stop=["The end", "The rest is for tomorrow."], # Stop generating just before the model would generate a new question
    echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion


In [ ]:
print(output_2)

## Translation

In [ ]:
# Translate story
prompt = output["choices"][0]['message']['content']

output_2 = llm.create_chat_completion( messages = [
        {"role": "system", "content": """
        You are a translation assistant. You translate the English input into German.
         """},
        {"role": "user", "content": output_1["choices"][0]['message']['content']}],
        temperature = 0.8, # Adjust the randomness of the generated text (default: 0.8).
        top_p = 0.90, # Limit the next token selection to a subset of tokens with a cumulative probability above a threshold P (default: 0.9).
        top_k = 40, # Limit the next token selection to the K most probable tokens (default: 40).
        min_p = 0.05, # https://github.com/ggerganov/llama.cpp/pull/3841 (default: 0.05)
        typical_p = 1.0, # https://arxiv.org/abs/2202.00666 (default: 1.0)
        repeat_penalty = 1.0 # The repeat-penalty option helps prevent the model from generating repetitive or monotonous text (default: 1.0, 1.0 = disabled).
        )

In [ ]:
print(output_2["choices"][0]['message']['content'])